In [ ]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai langchain-chroma bs4
%pip install -qU langchain-google-vertexai
%pip install pypdf

In [ ]:
# Import libraries:
# I'm following the LangChain documentation here: https://python.langchain.com/v0.2/docs/tutorials/qa_chat_history/

import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
import getpass
import os

# It will ask for your OpenAI key here

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Identical to what we already had:
# Read pdfs into the vector database

from langchain_community.document_loaders import PyPDFLoader

# def load_chunk_pdf(path= "../GenAI/"):
def load_chunk_pdf(path= "/content/gdrive/MyDrive/Colab Notebooks/Side Hustling 🤓"):
    pdf_folder_path = path
    documents = []
    for file in os.listdir(pdf_folder_path):
        if file.endswith('.pdf'):

            print (file)
            pdf_path = os.path.join(pdf_folder_path, file)
            loader = PyPDFLoader(pdf_path)
            documents.extend(loader.load())
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
    splits = text_splitter.split_documents(documents)
    vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

    return vectorstore

vectorstore = load_chunk_pdf()
retriever = vectorstore.as_retriever()

presente-indicativo.pdf
ACT.PRESENTE_V. IRREGULARES_CAMBIOS ORTOGRAFÍCOS.pdf
Presente de indicativo.pdf
ACT. VERBOS REFLEXIVOS.pdf
Verbos REFLEXIVOS y lista de más comunes.pdf
Los verbos reflexivos en español.pdf


In [ ]:
pip install gradio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 77.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.8/857.8 kB 50.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 87.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 16.9 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl size=5584 sha256=2e71694fd4012f014a14b0d36db093e03a962f8fd6110184342a57219ca46c1a
  Stored in directory: /root/.cache/pip/wheels/bd/65/9a/671fc6dcde07d4418df0c592f8df512b26d7a0029c2a23dd81
Successfully built ffmpy
  Attempting uninstall: websockets
    Found existing installation: websockets 12.0
    Uninstalling websockets-12.0:
      Successfully uninstalled websockets-12.0
  Attempting uninstall: altair
    Found existing installation: altair 4.2.2
    Uninstalling altair-4.2

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate

# Here you can fine tune your Prompt.

system_prompt = (
    "You are a friendly, witty, and informed professor chatbot. Your name is Uno. You are conversational. Start the conversation by asking them their name and which language they are learning."\
    "You only answer questions regarding Spanish to start. You can help with understanding nuances of Spain's Spanish vs. Latin American Spanish. You specialize in nuances across Spain, Argentina, Mexico, Chile, and Colombia."\
    "If you are asked a question about any other topic or language, you politely explain that you are not able to go beyond Spanish quite yet, but coming soon."\
    "You can help not only with grammar and culturally relevant language, terms, phrases but you also help them understand regionally specific slang."\
    "Give the user in-depth answers, providing some relevant facts, all that are culturally relevant."\
     "Act like a professor - quiz the user, have the user fill in the blank, play learning games back and forth. Be an engaging professor that is actually helping them learn."\
    "Finish your answer checking if whoever asked the question understood the answer or they need more help"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [ ]:
## Adding chat history

# This allows to make back references to previous questions, so Cass' answers will have "memory"

from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "\
    "which might reference context in the chat history, "\
    "formulate a standalone question which can be understood "\
    "without the chat history. Do NOT answer the question, "\
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [ ]:
# Adding conversation sessions to save and continue conversations

from langchain_core.messages import AIMessage, HumanMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [ ]:
# import gradio as gr

# gr.ChatInterface(get_llama_response).launch()

In [ ]:
# User interface with Gradio
# Change debug --> False and comment "prints"

import gradio as gr

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")
    chat_history = []

    def user (user_message, history):
        #print("user", user)
        #print("history", history)
        response= conversational_rag_chain.invoke(
            {"input": user_message},
            config={"configurable": {"session_id": "Test"}})
        #print(response['answer'])
        history.append((user_message, response["answer"]))
        #print("Updated chat history", history)
        return gr.update(value=""), history

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue= False)
    clear.click(lambda: None, None, chatbot, queue= False)

if __name__ == "__main__":
    demo.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://cf53463dfde95e7019.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Building the front end display